# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'09-18-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'09-18-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-09-19 04:22:56,33.93911,67.709953,38883,1437,32576,4870.0,Afghanistan,99.883527,3.695702
1,NaN,NaN,NaN,Albania,2020-09-19 04:22:56,41.15330,20.168300,12073,353,6831,4889.0,Albania,419.521857,2.923880
2,NaN,NaN,NaN,Algeria,2020-09-19 04:22:56,28.03390,1.659600,49413,1659,34818,12936.0,Algeria,112.683751,3.357416
3,NaN,NaN,NaN,Andorra,2020-09-19 04:22:56,42.50630,1.521800,1564,53,1164,347.0,Andorra,2024.202420,3.388747
4,NaN,NaN,NaN,Angola,2020-09-19 04:22:56,-11.20270,17.873900,3848,147,1443,2258.0,Angola,11.708053,3.820166


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38544,38572,38606,38641,38716,38772,38815,38855,38872,38883
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,10704,10860,11021,11185,11353,11520,11672,11816,11948,12073
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,47216,47488,47752,48007,48254,48496,48734,48966,49194,49413


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1420,1420,1420,1420,1420,1425,1426,1436,1436,1437
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,322,324,327,330,334,338,340,343,347,353
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1581,1591,1599,1605,1612,1620,1632,1645,1654,1659


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,31048,31129,31154,31234,31638,32073,32098,32503,32505,32576
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6284,6346,6443,6494,6569,6615,6668,6733,6788,6831
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,33379,33562,33723,33875,34037,34204,34385,34517,34675,34818


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1398,1413,1420,1432,1442,1447,1463,1619,1624,1664
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,4639,4678,4722,4752,4781,4800,4812,5003,5021,5033
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,618,618,618,620,622,626,629,809,809,824


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,23,23,23,23,23,23,23,24,24,24
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,42,42,43,43,43,43,43,47,48,48
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-09-19 04:22:56,32.539527,-86.644082,1664,24,0,1640.0,"Autauga, Alabama, US",2978.395890,1.442308
668,1075.0,Lamar,Alabama,US,2020-09-19 04:22:56,33.779950,-88.096680,337,2,0,335.0,"Lamar, Alabama, US",2441.144513,0.593472
669,1077.0,Lauderdale,Alabama,US,2020-09-19 04:22:56,34.901719,-87.656247,1616,33,0,1583.0,"Lauderdale, Alabama, US",1742.712636,2.042079


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,6723933,198570,2556465
India,5214677,84372,4112551
Brazil,4495183,135793,3897539


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,6723933,198570,2556465,3968898,2020-09-19 04:22:56,37.923092,-91.345036
India,5214677,84372,4112551,1017754,2020-09-19 04:22:56,23.088275,81.806127
Brazil,4495183,135793,3897539,461851,2020-09-19 04:22:56,-12.669522,-48.480493
Russia,1086955,19128,898420,169407,2020-09-19 04:22:56,54.546312,62.120860
Colombia,750471,23665,615457,111349,2020-09-19 04:22:56,5.349139,-74.190797


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,779491,14893,0
Texas,704563,14951,0
Florida,677660,13225,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,779491,14893,0,764598,2020-09-19 04:22:56,37.843962,-120.728594
Texas,704563,14951,0,689612,2020-09-19 04:22:56,31.660643,-98.653069
Florida,677660,13225,0,664435,2020-09-19 04:22:56,28.940755,-82.700744
New York,448052,33085,0,414967,2020-09-19 04:22:56,42.544151,-75.474183
Georgia,302854,6536,0,296318,2020-09-19 04:22:56,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,258516,6330,0
Florida,Miami-Dade,165980,3027,0
Arizona,Maricopa,139245,3245,0
Illinois,Cook,137580,5150,0
Texas,Harris,122445,2483,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,258516,6330,0,252186,2020-09-19 04:22:56,34.308284,-118.228241,6037.0
Florida,Miami-Dade,165980,3027,0,162953,2020-09-19 04:22:56,25.611236,-80.551706,12086.0
Arizona,Maricopa,139245,3245,0,136000,2020-09-19 04:22:56,33.348359,-112.491815,4013.0
Illinois,Cook,137580,5150,0,132430,2020-09-19 04:22:56,41.841448,-87.816588,17031.0
Texas,Harris,122445,2483,0,119962,2020-09-19 04:22:56,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-16,38855,11816,48966,1483,3675,95,589012,46376,26813,35073,...,380677,1856,49015,63416,1063,33006,10,2019,13887,7598
2020-09-17,38872,11948,49194,1483,3789,95,601713,46671,26861,35853,...,384087,1876,49627,64284,1066,33843,10,2022,13928,7633
2020-09-18,38883,12073,49413,1564,3848,95,613658,46910,26885,36661,...,388416,1890,50253,65174,1068,34401,10,2024,14022,7647


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-16,1436,343,1645,53,143,3,12116,923,832,758,...,41773,45,407,511,35,243,1,583,326,224
2020-09-17,1436,347,1654,53,144,3,12460,925,837,758,...,41794,46,413,520,35,244,1,585,326,224
2020-09-18,1437,353,1659,53,147,3,12656,926,844,763,...,41821,46,419,530,35,250,1,585,329,224


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-16,32503,6733,34517,1054,1401,91,448263,42069,23790,27655,...,2190,1559,45422,51274,936,22209,8,1221,12869,5823
2020-09-17,32505,6788,34675,1054,1405,91,456347,42231,23852,28044,...,2201,1582,45970,52564,940,23060,8,1221,13029,5841
2020-09-18,32576,6831,34818,1164,1443,92,467286,42369,23859,28451,...,2213,1603,46527,54218,941,23333,8,1221,13207,5883


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1398,1413,1420,1432,1442,1447,1463,1619,1624,1664
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,282,283,288,289,292,295,299,327,330,337
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1490,1504,1512,1518,1527,1530,1539,1590,1592,1616
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,510,515,526,529,532,534,546,573,576,578
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,3553,3560,3585,3593,3613,3622,3624,5570,5582,5603


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-16,141087,6432,0,209907,72219,771321,62666,55166,19234,49,...,177087,696807,59747,1702,1232,136269,80812,13202,92159,4566
2020-09-17,141757,6537,0,211660,73211,775037,63125,55386,19318,49,...,178140,701350,60658,1705,1238,137367,81198,13434,93819,4652
2020-09-18,142863,6644,0,212942,74082,779491,63730,55527,19366,49,...,180497,704563,61775,1706,1257,138605,81602,13685,97279,4747


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-16        1619    5003     809  612   1487     583    901    3165   
2020-09-17        1624    5021     809  617   1504     585    902    3211   
2020-09-18        1664    5033     824  619   1527     585    906    3249   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-16         1107      526  ...     196     16      224       65   
2020-09-17         1106      528  ...     199     18      226       67   
2020-09-18         1117      548  ...     202     18      231       70   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-16            317   481   323          0      111     23  
2020-09-17            320   487   327          0      112     23  
2020-09-18            322   490   327          1      113     23  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-16,2392,44,0,5371,1157,14691,2002,4487,619,0,...,2151,14738,437,58,19,2882,2020,293,1227,50
2020-09-17,2401,44,0,5409,1166,14804,2006,4488,619,0,...,2164,14826,437,58,19,2918,2031,297,1230,49
2020-09-18,2428,45,0,5451,1173,14893,2009,4492,620,0,...,2196,14951,437,58,19,2947,2037,300,1238,49


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-16          24      47       7    9     13      14     38      38   
2020-09-17          24      48       7    9     13      14     38      39   
2020-09-18          24      48       7   10     14      14     38      39   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-16           42       15  ...       1      1        4        1   
2020-09-17           42       16  ...       2      1        4        1   
2020-09-18           42       16  ...       2      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-16              2     1     2          4        6      0  
2020-09-17              2     1     2          0        6      0  
2020-09-18              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-16,0.001031,0.012337,0.004761,0.031293,0.029700,0.0,0.020220,0.005573,0.001307,0.022387,...,0.010638,0.015873,0.012100,0.012146,0.000000,0.023442,0.0,0.001488,0.004921,0.002904
2020-09-17,0.000438,0.011171,0.004656,0.000000,0.031020,0.0,0.021563,0.006361,0.001790,0.022239,...,0.008958,0.010776,0.012486,0.013687,0.002822,0.025359,0.0,0.001486,0.002952,0.004606
2020-09-18,0.000283,0.010462,0.004452,0.054619,0.015571,0.0,0.019852,0.005121,0.000893,0.022536,...,0.011271,0.007463,0.012614,0.013845,0.001876,0.016488,0.0,0.000989,0.006749,0.001834


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-16,0.007013,0.008824,0.007966,0.0,0.028777,0.0,0.022275,0.003261,0.009709,0.001321,...,0.000479,0.000000,0.012438,0.017928,0.0,0.061135,0.0,0.000000,0.006173,0.0
2020-09-17,0.000000,0.011662,0.005471,0.0,0.006993,0.0,0.028392,0.002167,0.006010,0.000000,...,0.000503,0.022222,0.014742,0.017613,0.0,0.004115,0.0,0.003431,0.000000,0.0
2020-09-18,0.000696,0.017291,0.003023,0.0,0.020833,0.0,0.015730,0.001081,0.008363,0.006596,...,0.000646,0.000000,0.014528,0.019231,0.0,0.024590,0.0,0.000000,0.009202,0.0


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-16,0.012618,0.009748,0.003839,0.115344,0.051802,0.000000,0.021372,0.003052,0.005920,0.011004,...,0.002747,0.009061,0.010680,0.018129,0.005371,0.018575,0.0,0.001641,0.022160,0.006917
2020-09-17,0.000062,0.008169,0.004577,0.000000,0.002855,0.000000,0.018034,0.003851,0.002606,0.014066,...,0.005023,0.014753,0.012065,0.025159,0.004274,0.038318,0.0,0.000000,0.012433,0.003091
2020-09-18,0.002184,0.006335,0.004124,0.104364,0.027046,0.010989,0.023971,0.003268,0.000293,0.014513,...,0.005452,0.013274,0.012117,0.031466,0.001064,0.011839,0.0,0.000000,0.013662,0.007191


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-16,0.006614,0.007519,NaN,0.003336,0.010098,0.004058,0.009456,0.002453,0.005069,0.0,...,0.010592,0.005852,0.012661,0.000000,0.000000,0.006210,0.004312,0.016633,0.012047,0.028378
2020-09-17,0.004749,0.016325,NaN,0.008351,0.013736,0.004818,0.007325,0.003988,0.004367,0.0,...,0.005946,0.006520,0.015248,0.001763,0.004870,0.008058,0.004777,0.017573,0.018012,0.018835
2020-09-18,0.007802,0.016368,NaN,0.006057,0.011897,0.005747,0.009584,0.002546,0.002485,0.0,...,0.013231,0.004581,0.018415,0.000587,0.015347,0.009012,0.004975,0.018684,0.036880,0.020421


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-16      0.106630  0.039692  0.286169  0.055172  0.305531  0.044803   
2020-09-17      0.003088  0.003598  0.000000  0.008170  0.011432  0.003431   
2020-09-18      0.024631  0.002390  0.018541  0.003241  0.015293  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-16      0.038018  0.128745  0.279769  0.286064  ...  0.048128   
2020-09-17      0.001110  0.014534 -0.000903  0.003802  ...  0.015306   
2020-09-18      0.004435  0.011834  0.009946  0.037879  ...  0.015075   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-16      0.333333  0.051643  0.048387   0.000000  0.006276  0.035256   
2020-09-17      0.125000  0.008929  0.030769   0.009464  0.012474  0.012384   
2020-09-18      0.000000  0.022124  0.044776   0.006250  0.006160  0.000000   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2020-09-16           -1.0  0.000000    0.0  
2020-09-17            NaN  0.009009    0.0  
2020-09-18            inf  0.008929    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-16,0.002095,0.000000,NaN,0.005052,0.006087,0.006647,0.003006,0.000446,0.001618,NaN,...,0.011283,0.009936,0.002294,0.0,0.0,0.015862,0.002481,0.035336,0.005738,0.190476
2020-09-17,0.003763,0.000000,NaN,0.007075,0.007779,0.007692,0.001998,0.000223,0.000000,NaN,...,0.006044,0.005971,0.000000,0.0,0.0,0.012491,0.005446,0.013652,0.002445,-0.020000
2020-09-18,0.011245,0.022727,NaN,0.007765,0.006003,0.006012,0.001496,0.000891,0.001616,NaN,...,0.014787,0.008431,0.000000,0.0,0.0,0.009938,0.002954,0.010101,0.006504,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour      Bibb    Blount   Bullock   
2020-09-16      0.043478  0.093023     0.0  0.500000  0.000000  0.076923   
2020-09-17      0.000000  0.021277     0.0  0.000000  0.000000  0.000000   
2020-09-18      0.000000  0.000000     0.0  0.111111  0.076923  0.000000   

Province_State                                         ... Wyoming         \
Admin2            Butler   Calhoun Chambers  Cherokee  ...    Park Platte   
2020-09-16      0.027027  0.225806     0.05  0.363636  ...     0.0    0.0   
2020-09-17      0.000000  0.026316     0.00  0.066667  ...     1.0    0.0   
2020-09-18      0.000000  0.000000     0.00  0.000000  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2020-09-16          3.0      0.0        0.0   0.0   0.0        inf      0.0   
2020-09-17          0.0      0.0        0.0   0.0   0.0       -1.0      0.0   
2020-09-18          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2020-09-16        NaN  
2020-09-17        NaN  
2020-09-18        NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-16,0.001150,0.013167,0.004902,0.025069,0.028456,6.493308e-07,0.019968,0.004386,0.001443,0.020645,...,0.009432,0.011698,0.012156,0.014619,0.000220,0.024810,7.180387e-28,0.001398,0.006876,0.003495
2020-09-17,0.000794,0.012169,0.004779,0.012535,0.029738,3.246654e-07,0.020765,0.005373,0.001617,0.021442,...,0.009195,0.011237,0.012321,0.014153,0.001521,0.025085,3.590194e-28,0.001442,0.004914,0.004051
2020-09-18,0.000538,0.011316,0.004615,0.033577,0.022655,1.623327e-07,0.020309,0.005247,0.001255,0.021989,...,0.010233,0.009350,0.012468,0.013999,0.001699,0.020786,1.795097e-28,0.001216,0.005832,0.002942


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-16,0.004135,0.008675,0.007018,2.798445e-13,0.023484,2.206635e-47,0.020104,0.002806,0.008337,0.001227,...,0.000450,0.000003,0.014016,0.017535,2.427385e-06,0.041773,0.0,0.000228,0.010400,0.000307
2020-09-17,0.002067,0.010168,0.006244,1.399223e-13,0.015238,1.103317e-47,0.024248,0.002486,0.007173,0.000614,...,0.000476,0.011113,0.014379,0.017574,1.213693e-06,0.022944,0.0,0.001829,0.005200,0.000153
2020-09-18,0.001382,0.013730,0.004634,6.996113e-14,0.018036,5.516586e-48,0.019989,0.001784,0.007768,0.003605,...,0.000561,0.005556,0.014453,0.018402,6.068463e-07,0.023767,0.0,0.000915,0.007201,0.000077


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-16,0.009209,0.009116,0.004463,0.058046,0.030686,8.578526e-08,0.022351,0.003515,0.004995,0.010942,...,0.014935,0.009383,0.011343,0.018235,0.006335,0.024001,4.207258e-30,0.002045,0.018217,0.008202
2020-09-17,0.004635,0.008642,0.004520,0.029023,0.016771,4.289263e-08,0.020192,0.003683,0.003801,0.012504,...,0.009979,0.012068,0.011704,0.021697,0.005304,0.031160,2.103629e-30,0.001023,0.015325,0.005646
2020-09-18,0.003410,0.007489,0.004322,0.066694,0.021908,5.494527e-03,0.022082,0.003475,0.002047,0.013509,...,0.007716,0.012671,0.011910,0.026582,0.003184,0.021499,1.051815e-30,0.000511,0.014494,0.006419


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-16,0.006159,0.008487,NaN,0.002652,0.010492,0.004141,0.007948,0.003277,0.006621,1.388363e-56,...,0.009253,0.006906,0.011231,0.002306,0.003144,0.006525,0.004311,0.014807,0.012270,0.019560
2020-09-17,0.005454,0.012406,NaN,0.005502,0.012114,0.004479,0.007636,0.003632,0.005494,6.941813e-57,...,0.007600,0.006713,0.013239,0.002034,0.004007,0.007291,0.004544,0.016190,0.015141,0.019197
2020-09-18,0.006628,0.014387,NaN,0.005779,0.012006,0.005113,0.008610,0.003089,0.003989,3.470907e-57,...,0.010415,0.005647,0.015827,0.001310,0.009677,0.008152,0.004760,0.017437,0.026010,0.019809


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-16      0.057424  0.021805  0.145396  0.028422  0.156835  0.023218   
2020-09-17      0.030256  0.012702  0.072698  0.018296  0.084134  0.013324   
2020-09-18      0.027443  0.007546  0.045620  0.010769  0.049713  0.006662   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-16      0.019617  0.068315  0.140435  0.147964  ...  0.025701   
2020-09-17      0.010363  0.041425  0.069766  0.075883  ...  0.020504   
2020-09-18      0.007399  0.026630  0.039856  0.056881  ...  0.017789   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-16      0.235020  0.036280  0.063150  -0.000599  0.007036  0.018278   
2020-09-17      0.180010  0.022604  0.046959   0.004432  0.009755  0.015331   
2020-09-18      0.090005  0.022364  0.045868   0.005341  0.007958  0.007666   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-09-16      -0.571423  0.000006  0.003197  
2020-09-17      -0.571423  0.004507  0.001598  
2020-09-18      -0.571423  0.006718  0.000799  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-16,0.005202,0.001113,NaN,0.003907,0.045975,0.007256,0.002496,0.000379,0.001878,NaN,...,0.012018,0.008772,0.002289,4.020733e-15,0.001887,0.017614,0.003430,0.028329,0.005473,0.095244
2020-09-17,0.004482,0.000557,NaN,0.005491,0.026877,0.007474,0.002247,0.000301,0.000939,NaN,...,0.009031,0.007371,0.001144,2.010366e-15,0.000943,0.015053,0.004438,0.020990,0.003959,0.037622
2020-09-18,0.007864,0.011642,NaN,0.006628,0.016440,0.006743,0.001871,0.000596,0.001277,NaN,...,0.011909,0.007901,0.000572,1.005183e-15,0.000472,0.012496,0.003696,0.015546,0.005232,0.018811


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-09-16      0.021740  0.046893  3.118965e-10  0.249993  0.002617   
2020-09-17      0.010870  0.034085  1.559482e-10  0.124996  0.001308   
2020-09-18      0.005435  0.017043  7.797412e-11  0.118054  0.039116   

Province_State                                                    ...  \
Admin2           Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-09-16      0.038462  0.013517  0.112903  0.025003  0.181818  ...   
2020-09-17      0.019231  0.006758  0.069609  0.012502  0.124242  ...   
2020-09-18      0.009615  0.003379  0.034805  0.006251  0.062121  ...   

Province_State   Wyoming                                                   \
Admin2              Park Platte  Sheridan Sublette Sweetwater Teton Uinta   
2020-09-16      0.000000    0.0  1.500092      0.0        0.0   0.0   0.0   
2020-09-17      0.500015    0.0  0.750023      0.0        0.0   0.0   0.0   
2020-09-18      0.250004    0.0  0.375006      0.0        0.0   0.0   0.0   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-09-16      -0.499985      0.0    NaN  
2020-09-17      -0.999969      0.0    NaN  
2020-09-18      -0.999969      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200918,AK,7587,410706,NaN,418293,36.0,NaN,NaN,NaN,...,418293,1,0,cc5fc47f4be842f963af84b7fcd10209b229c2b9,0,0,0,0,0,NaN
1,20200918,AL,142863,910192,NaN,1039010,744.0,16227.0,NaN,1638.0,...,1053055,27,148,1d80c9982122b8a5e07be65d84c08ac3b4d4e74e,0,0,0,0,0,NaN
2,20200918,AR,74082,794097,NaN,866435,372.0,4896.0,193.0,NaN,...,868179,7,0,f2a66b16a735419549a96a760bcf829a8594a586,0,0,0,0,0,NaN
3,20200918,AS,0,1571,NaN,1571,NaN,NaN,NaN,NaN,...,1571,0,0,4953337207aa6ea9f70d3e91b93db3bd67bd2589,0,0,0,0,0,NaN
4,20200918,AZ,212942,1149377,NaN,1362319,516.0,21865.0,130.0,NaN,...,1362319,42,44,a326957ad5bbabb2b7d4f6d97e63c725b2420f9c,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200918,AK,7587.0,410706.0,NaN,418293.0,36.0,NaN,NaN,NaN,...,418293,1,0,cc5fc47f4be842f963af84b7fcd10209b229c2b9,0,0,0,0,0,NaN
1,20200918,AL,142863.0,910192.0,NaN,1039010.0,744.0,16227.0,NaN,1638.0,...,1053055,27,148,1d80c9982122b8a5e07be65d84c08ac3b4d4e74e,0,0,0,0,0,NaN
2,20200918,AR,74082.0,794097.0,NaN,866435.0,372.0,4896.0,193.0,NaN,...,868179,7,0,f2a66b16a735419549a96a760bcf829a8594a586,0,0,0,0,0,NaN
3,20200918,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,...,1571,0,0,4953337207aa6ea9f70d3e91b93db3bd67bd2589,0,0,0,0,0,NaN
4,20200918,AZ,212942.0,1149377.0,NaN,1362319.0,516.0,21865.0,130.0,NaN,...,1362319,42,44,a326957ad5bbabb2b7d4f6d97e63c725b2420f9c,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-18,AK,7587,410706,NaN,418293,36.0,NaN,NaN,NaN,13.0,...,418293,1,0,cc5fc47f4be842f963af84b7fcd10209b229c2b9,0,0,0,0,0,NaN
2020-09-18,AL,142863,910192,NaN,1039010,744.0,16227.0,NaN,1638.0,NaN,...,1053055,27,148,1d80c9982122b8a5e07be65d84c08ac3b4d4e74e,0,0,0,0,0,NaN
2020-09-18,AR,74082,794097,NaN,866435,372.0,4896.0,193.0,NaN,74.0,...,868179,7,0,f2a66b16a735419549a96a760bcf829a8594a586,0,0,0,0,0,NaN
2020-09-18,AS,0,1571,NaN,1571,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,4953337207aa6ea9f70d3e91b93db3bd67bd2589,0,0,0,0,0,NaN
2020-09-18,AZ,212942,1149377,NaN,1362319,516.0,21865.0,130.0,NaN,72.0,...,1362319,42,44,a326957ad5bbabb2b7d4f6d97e63c725b2420f9c,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-18,AK,7587.0,410706.0,NaN,418293.0,36.0,NaN,NaN,NaN,13.0,...,418293,1,0,cc5fc47f4be842f963af84b7fcd10209b229c2b9,0,0,0,0,0,NaN
2020-09-18,AL,142863.0,910192.0,NaN,1039010.0,744.0,16227.0,NaN,1638.0,NaN,...,1053055,27,148,1d80c9982122b8a5e07be65d84c08ac3b4d4e74e,0,0,0,0,0,NaN
2020-09-18,AR,74082.0,794097.0,NaN,866435.0,372.0,4896.0,193.0,NaN,74.0,...,868179,7,0,f2a66b16a735419549a96a760bcf829a8594a586,0,0,0,0,0,NaN
2020-09-18,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,4953337207aa6ea9f70d3e91b93db3bd67bd2589,0,0,0,0,0,NaN
2020-09-18,AZ,212942.0,1149377.0,NaN,1362319.0,516.0,21865.0,130.0,NaN,72.0,...,1362319,42,44,a326957ad5bbabb2b7d4f6d97e63c725b2420f9c,0,0,0,0,0,NaN
